### Content workflow
1. Loading and preprocessing the data
2. Initializing a language model and extracting a knowledge graph
3. Storing the knowledge graph in a database for persistence and querying.
4. Retrieving relevant knowledge from the graph based on user queries.
5. Generating responses by synthesizing the retrieved knowledge with the language model.


#### importing  librairies

In [96]:

import os

from langchain.document_loaders import TextLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.llms import OpenAI
from langchain_openai import ChatOpenAI
from langchain_experimental.graph_transformers import LLMGraphTransformer
from dotenv import load_dotenv

from llama_index.core import KnowledgeGraphIndex, SimpleDirectoryReader

load_dotenv()


True

#### Loading the text document

In [81]:

# Load text data

loader = TextLoader("./text.txt")
documents = loader.load()

text_splitter = CharacterTextSplitter(chunk_size=200, chunk_overlap=20)
texts = text_splitter.split_documents(documents)


In [82]:
# defining the llm transformer model and the graph model object 
llm = ChatOpenAI(temperature=0, model_name="gpt-4-turbo")
llm_transformer = LLMGraphTransformer(llm=llm)


graph_documents = llm_transformer.convert_to_graph_documents(texts)

In [83]:
print(f"Nodes: {graph_documents[0].nodes}")
print(f"Edges: {graph_documents[0].relationships}")

Nodes: [Node(id='Sarah', type='Person'), Node(id='Prismaticai', type='Company'), Node(id='Michael', type='Person')]
Edges: [Relationship(source=Node(id='Sarah', type='Person'), target=Node(id='Prismaticai', type='Company'), type='EMPLOYEE'), Relationship(source=Node(id='Michael', type='Person'), target=Node(id='Prismaticai', type='Company'), type='EMPLOYEE')]


#### Store knowledge graph in a database

In [97]:
from langchain_community.graphs import Neo4jGraph

url = os.environ["NEO4J_URL"]
username = os.environ["NEO_4J_USERNAME"]
password = os.environ["NEO_4J_PASSWORD"]

graph_store = Neo4jGraph(url=url, username=username, password=password)


graph_store.add_graph_documents(graph_documents)

In [89]:
#### Inserting new information to the graph
graph_store.query(
    """
    MERGE (m:Movie {title:"Top Gun"})
    WITH m
    UNWIND ["Tom Cruise", "Val Kilmer", "Anthony Edwards", "Meg Ryan"] AS actor
    MERGE (a:Actor {name:actor})
    MERGE (a)-[:ACTED_IN]->(m)
    """
)

[]

#### Quering and retrieving data from the graph store

In [94]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    ChatOpenAI(temperature=0), graph=graph_store, verbose=True
)

# chain.run("Where does Sarah works?")
chain.run("Where does Sarah works?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (p:Person {id: 'Sarah'})-[:EMPLOYEE]->(c:Company)
RETURN c;
Full Context:
[{'c': {'id': 'Prismaticai'}}]

> Finished chain.


'Sarah works at Prismaticai.'